In [1]:
import requests
import json
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

PATH = "/home/kasia/code/chromedriver"




In [63]:
def scrap_product(article_code, item_count):
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('--window-size=1920x1080') 

    driver = webdriver.Chrome(executable_path=PATH,options=chrome_options)
    driver.get(f"https://www.ikea.com/de/de/search/products/?q={article_code}")
    X = driver.find_element_by_class_name('results__list')
    tag = X.find_element_by_tag_name('a')
    #https://stackoverflow.com/questions/48665001/can-not-click-on-a-element-elementclickinterceptedexception-in-splinter-selen
    driver.execute_script("arguments[0].click();", tag)
    #print(tag)
    
    #go to product page
    #tag.click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    X = soup.find_all("div",  {"id": 'SEC_product-details-packaging'})
    return X[0]


def extract_numeric_product_to_list(y):
    
    """
    ['Artikelnummer:',
     'Breite: 61 cm',
     'Höhe: 7 cm',
     'Länge: 245 cm',
     'Gewicht: 44.16 kg',
     'Paket(e): 2']
    """
    numeric_product_info = []
    for x in [item for subitem in y for item in subitem.split()]:   
        try:
            
            x = float(x) 
            numeric_product_info.append(x)       
        except (ValueError, TypeError):
            pass
    return numeric_product_info

def extract_numeric_product_to_dict(y):
    info_dict = {}
    new_columns_name = ['width','high','length','weight','packeges']
    for info in y:
        info_item = info.split()
        for i,x in enumerate(info_item):            
            try:
                float(x)
                info_dict[info_item[0]] = float(info_item[1])          
            except:
                pass
    info_dict = {x:y for x,y in zip(new_columns_name,info_dict.values())}
    

    return info_dict

def packages_dimensions_weights(page):
    info = page.find_all('div',  {"class": 'range-revamp-product-details__container'})
    number = page.find_all('span',  {"class": 'range-revamp-product-identifier__value'})

    list_of_products = []
    product_info = {}
    for i,(x,y) in enumerate(zip(number,info)):
        y_info = [info.text for info in y.find_all('span',  {"class": 'range-revamp-product-details__label'})]
        product_info = extract_numeric_product_to_dict(y_info) 
        product_info['subarticle_code'] = x.text
        
        list_of_products.append(product_info)
    return pd.DataFrame(list_of_products)

def rename_columns(df):
    new_columns_name = ['width','high','length','weight','packeges','subarticle_code', 'article_code']
    rename_columns = {x:y for x,y in zip(df.columns,new_columns_name)}
    df.rename(columns = rename_columns, inplace = True)
    
    return df

def product_info_and_update_csv_database(article_code,path_to_csv):
    
    ikea_database = pd.read_csv(path_to_csv,index_col = [0])
    all_ordered_product_df = pd.DataFrame()
    new_product_for_database = pd.DataFrame()
    for i,x in enumerate(article_code):
        if ikea_database.shape[0]>0 and (ikea_database['article_code'] == x).any():
            all_ordered_product_df = all_ordered_product_df.append(ikea_database[ikea_database['article_code'] == x])
        else:
            page = scrap_product(x,item_count=None)
            df = packages_dimensions_weights(page)
            df['article_code'] = article_code[i]

            all_ordered_product_df = all_ordered_product_df.append(df)
            new_product_for_database = new_product_for_database.append(df)

    ikea_database = ikea_database.append(new_product_for_database)
    ikea_database.to_csv(path_to_csv)
    return all_ordered_product_df

In [70]:
df = pd.DataFrame(columns = ['width','high','length','weight','packeges','subarticle_code', 'article_code'])


df


,width,high,length,weight,packeges,subarticle_code,article_code


In [166]:

article_code = ['s39128564','90499066','s29178074']
path_to_csv = 'ikea_database.csv'
product_info_and_update_csv_database(article_code,path_to_csv)    
    
    


,width,high,length,weight,packeges,subarticle_code,article_code
0,6.0,4.0,49.0,0.28,1.0,902.568.93,s39128564
1,6.0,4.0,49.0,0.28,1.0,402.568.95,s39128564
2,6.0,4.0,74.0,0.43,1.0,302.779.59,s39128564
3,6.0,4.0,74.0,0.43,1.0,802.145.68,s39128564
4,48.0,2.0,59.0,3.35,6.0,202.145.71,s39128564
5,48.0,2.0,59.0,3.35,6.0,102.463.32,s39128564
6,61.0,7.0,245.0,44.16,2.0,902.779.61,s39128564
0,78.0,65.0,96.0,41.20,1.0,904.990.66,90499066
0,58.0,55.0,59.0,6.70,1.0,003.204.93,s29178074
1,58.0,55.0,59.0,6.70,1.0,803.204.94,s29178074


In [167]:
pd.read_csv('ikea_database.csv',index_col=[0])


,width,high,length,weight,packeges,subarticle_code,article_code
0,6.0,4.0,49.0,0.28,1.0,902.568.93,s39128564
1,6.0,4.0,49.0,0.28,1.0,402.568.95,s39128564
2,6.0,4.0,74.0,0.43,1.0,302.779.59,s39128564
3,6.0,4.0,74.0,0.43,1.0,802.145.68,s39128564
4,48.0,2.0,59.0,3.35,6.0,202.145.71,s39128564
5,48.0,2.0,59.0,3.35,6.0,102.463.32,s39128564
6,61.0,7.0,245.0,44.16,2.0,902.779.61,s39128564
0,78.0,65.0,96.0,41.20,1.0,904.990.66,90499066
0,58.0,55.0,59.0,6.70,1.0,003.204.93,s29178074
1,58.0,55.0,59.0,6.70,1.0,803.204.94,s29178074


## Draft



In [5]:
def extract_numeric_product_to_list(y):
    """
    ['Artikelnummer:',
     'Breite: 61 cm',
     'Höhe: 7 cm',
     'Länge: 245 cm',
     
     'Gewicht: 44.16 kg',
     'Paket(e): 2']
    """
    numeric_product_info = []
    for x in [item for subitem in y for item in subitem.split()]:   
        try:
            x = float(x) 
            numeric_product_info.append(x)       
        except (ValueError, TypeError):
            pass
    return numeric_product_info

def packages_dimensions_weights(page):
    """
    [(
    article_code (str),
    item_count (int),
    [(
        package_id (int),
        package_length (int),
        package_width (int),
        package_height (int),
        package_weight (float)
    )])]
    """
    info = page.find_all('div',  {"class": 'range-revamp-product-details__container'})
    number = page.find_all('span',  {"class": 'range-revamp-product-identifier__value'})
    list_info = []
    for i,(x,y) in enumerate(zip(number,info),start = 1):
        y = [info.text for info in y.find_all('span',  {"class": 'range-revamp-product-details__label'})]
        list_info.append([(x.text,i),extract_numeric_product_to_list(y)])  
    return list_info
    
#packages_dimensions_weights(page)

def products_info_to_df(article_code,path_to_csv):
    
    all_ordered_product_df = pd.DataFrame()
    for i,x in enumerate(article_code):
    
        page = scrap_product(x,item_count=None)

        df = packages_dimensions_weights(page)
        df['article_code'] = article_code[i]
        all_ordered_product_df = all_ordered_product_df.append(df)
        
    
    return all_ordered_product_df


In [27]:
empoyees = [('jack', 34, 'Sydney', 155) ,
            ('Riti', 31, 'Delhi' , 177) ,('jack', 348, 'Sydney', 155) ]

d = pd.DataFrame(empoyees)
d.columns=['a','b','c','d']
'jack' in d[0]



index = d.index
condition = d[0] == "jack"
d[condition]



KeyError: 0

In [25]:
'jack' in d[0]


False